In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

In [0]:
import sys
# -----------------------------
# Ajouter le repo au Python Path
# -----------------------------
sys.path.append("/Workspace/Users/mandu543@gmail.com/databricks-ecommerce/Pipelines/")

from lib.utils import *

### Products

In [0]:
df_products = spark.table(f"{SILVER_ZONE}.slv_ecommerce_products")
df_brands = spark.table(f"{SILVER_ZONE}.slv_ecommerce_brands")
df_category = spark.table(f"{SILVER_ZONE}.slv_ecommerce_categories")

In [0]:
df_products.createOrReplaceTempView("v_products")
df_brands.createOrReplaceTempView("v_brands")
df_category.createOrReplaceTempView("v_category")

In [0]:
display(spark.sql("select * from v_products limit 5"))


In [0]:
display(spark.sql("select * from v_brands limit 10"))


In [0]:
display(spark.sql("select * from v_category limit 10"))

In [0]:
# Make sure we’re on the right catalog
spark.sql(f"USE CATALOG {CATALOG}")

In [0]:
%sql

-- Build brands×category mapping and write Gold table
CREATE OR REPLACE TABLE `03_gold`.dim_ecommerce_products AS

WITH brands_categories AS (
  SELECT
    b.brand_name,
    b.brand_code,
    c.category_name,
    c.category_code
  FROM v_brands b
  INNER JOIN v_category c
  ON
    b.category_code = c.category_code
)
SELECT
  p.product_id,
  p.sku,
  p.category_code,
  COALESCE(bc.category_name, 'Not Available') AS category_name,
  p.brand_code,
  COALESCE(bc.brand_name, 'Not Available')   AS brand_name,
  p.color,
  p.size,
  p.material,
  p.weight_grams,
  p.length_cm,
  p.width_cm,
  p.height_cm,
  p.rating_count,
  p._source_file ,
  p.date_creation,
  p.date_modification
FROM v_products p
LEFT JOIN brands_categories bc
  ON p.brand_code = bc.brand_code;


### Customers

On aimerait bien rajouter les difféntes regions dans la table customers

In [0]:
# India states grouped by region

india_region = {

    # West
    "MH": "West",   # Maharashtra
    "GJ": "West",   # Gujarat
    "RJ": "West",   # Rajasthan
    "GA": "West",   # Goa

    # South
    "KA": "South",  # Karnataka
    "TN": "South",  # Tamil Nadu
    "TS": "South",  # Telangana
    "AP": "South",  # Andhra Pradesh
    "KL": "South",  # Kerala

    # North
    "UP": "North",  # Uttar Pradesh
    "PB": "North",  # Punjab
    "HR": "North",  # Haryana
    "HP": "North",  # Himachal Pradesh
    "UK": "North",  # Uttarakhand
    "DL": "North",  # Delhi

    # East
    "WB": "East",   # West Bengal
    "BR": "East",   # Bihar
    "OD": "East",   # Odisha
    "JH": "East",   # Jharkhand

    # Northeast
    "AS": "Northeast",
    "AR": "Northeast",
    "MN": "Northeast",
    "ML": "Northeast",
    "MZ": "Northeast",
    "NL": "Northeast",
    "TR": "Northeast",
    "SK": "Northeast",

    # Central
    "MP": "Central",
    "CG": "Central"

}
 

# Australia states / territories grouped by region

australia_region = {

    "VIC": "SouthEast",   # Victoria
    "WA": "West",         # Western Australia
    "NSW": "East",        # New South Wales
    "QLD": "NorthEast",   # Queensland
    "SA": "South",        # South Australia
    "TAS": "South",       # Tasmania
    "ACT": "East",        # Australian Capital Territory
    "NT": "North"         # Northern Territory

}

# United Kingdom countries

uk_region = {

    "ENG": "England",              # England
    "WLS": "Wales",                # Wales
    "NIR": "Northern Ireland",     # Northern Ireland
    "SCT": "Scotland"              # Scotland

}

# United States states grouped by region

us_region = {

    "MA": "NorthEast",  # Massachusetts
    "FL": "South",      # Florida
    "NJ": "NorthEast",  # New Jersey
    "CA": "West",       # California
    "NY": "NorthEast",  # New York
    "TX": "South"       # Texas

}

# UAE emirates

uae_region = {

    "AUH": "Abu Dhabi",         # Abu Dhabi
    "DXB": "Dubai",             # Dubai
    "SHJ": "Sharjah",           # Sharjah
    "AJM": "Ajman",             # Ajman
    "RAK": "Ras Al Khaimah",    # Ras Al Khaimah
    "FUJ": "Fujairah",          # Fujairah
    "UAQ": "Umm Al Quwain"      # Umm Al Quwain


}

# Singapore (city-state)

singapore_region = {

    "SG": "Singapore"  # Singapore  

}

# Canada provinces grouped by region

canada_region = {

    "BC": "West",     # British Columbia
    "AB": "West",     # Alberta
    "SK": "West",     # Saskatchewan
    "MB": "Central",  # Manitoba
    "ON": "East",     # Ontario
    "QC": "East",     # Quebec
    "NB": "East",     # New Brunswick
    "PE": "East",     # Prince Edward Island
    "NS": "East"      # Nova Scotia

}
 

# Combine into a master dictionary
country_state_map = {
    "India": india_region,
    "Australia": australia_region,
    "United Kingdom": uk_region,
    "United States": us_region,
    "United Arab Emirates": uae_region,
    "Singapore": singapore_region,
    "Canada": canada_region
}  

In [0]:
from pyspark.sql import Row

# 1 Flatten country_state_map into a list of Rows
rows = []
for country, states in country_state_map.items():
    for state_code, region in states.items():
        rows.append(Row(country=country, state=state_code, region=region))
rows[:15]  

In [0]:
# 2️ Create mapping DataFrame
df_region_mapping = spark.createDataFrame(rows)

In [0]:
display(df_region_mapping)

In [0]:
df_silver_customers = spark.table(f'{SILVER_ZONE}.slv_ecommerce_customers')
display(df_silver_customers.limit(5))

In [0]:
# Join DF customers silvers and Region
df_gold_customers = df_silver_customers.join(df_region_mapping, on=['country', 'state'], how='left')
df_gold_customers = df_gold_customers.fillna({'region': 'Other'})

# Move all columns
priority_cols = [
    "customer_id",
    "phone",
    "country",
    "country_code",
    "state",
    "region",
    "date_creation",
    "date_modification"
]

# Ajouter les autres colonnes sans changer leur ordre
final_cols = priority_cols + [
    c for c in df_gold_customers.columns if c not in priority_cols
]

df_gold_customers = df_gold_customers.select(*final_cols)
display(df_gold_customers.limit(5))
 

In [0]:
df_gold_customers.write.format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable(f"{GOLD_ZONE}.dim_ecommerce_customers")